# More topics on Melbourne House Prices

This notebeook is a continuation of the notebook `house-pricing`. The idea is to go deep i nnew topics in ML.

First

## MISSING VALUES

Missing values are the most common data issue that you will be able to find in almost every data set. The best thing is to be prepared to face this challenge. 

Most of the machine learning libraries (like sci-kit learn) cannot deal with missing values. These raise an error if a model is built using data with missing values. To avoid these, there are some strategies.
 
**1. Drop columns with missing values**

This is the mos simple option but not the most recommended as it has to be clear that the column that is removed does not add anything to the model.

**2. Filling the missing values (Imputation)**

**Filling** the missing values with some other value. An example of a filling value is the _mean_ but in case you have [skewed](https://en.wikipedia.org/wiki/Skewness) data you might want to use the median, or even the mode. 

This is probably the best method that can be used. This depends on that the value that is added really makes sense as a value for the numerical values that are in the range of the column. 

**3. Extension of filling **

Filling is the standard approach, and it usually works well. There might be disadvantages as the value that is used to fill in the missing might not be completely accurate, or the rows with the missig values can be unique in some other way. In that case, the model would make better prediction considering which values were originlly missing. For that purpose, for each column with a missing value, a new column is added `"original_name"_was_missing` that has as values `True`if the value in the `original_name` was missing and `False` otherwise. This trick can help in some cases but is not always the case.

Now, let us test those three in the following.

In [4]:
# import all the necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

df = pd.read_csv('data/Melbourne_housing_FULL.csv')
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         34857 non-null  object 
 1   Address        34857 non-null  object 
 2   Rooms          34857 non-null  int64  
 3   Type           34857 non-null  object 
 4   Price          27247 non-null  float64
 5   Method         34857 non-null  object 
 6   SellerG        34857 non-null  object 
 7   Date           34857 non-null  object 
 8   Distance       34856 non-null  float64
 9   Postcode       34856 non-null  float64
 10  Bedroom2       26640 non-null  float64
 11  Bathroom       26631 non-null  float64
 12  Car            26129 non-null  float64
 13  Landsize       23047 non-null  float64
 14  BuildingArea   13742 non-null  float64
 15  YearBuilt      15551 non-null  float64
 16  CouncilArea    34854 non-null  object 
 17  Lattitude      26881 non-null  float64
 18  Longti

In [ ]:
# Case 1. drop the columns 